In [1]:
import numpy as np
import scipy
import matplotlib.pyplot as plt

# read image
import matplotlib.image as mpimg

# import tensorflow
import tensorflow as tf

print(tf.__version__)

2.10.0


In [2]:
# Get the list of available GPUs
gpu_devices = tf.config.experimental.list_physical_devices('GPU')

# Print the list of available GPUs
print("Available GPUs:", gpu_devices)

# Check if TensorFlow is using GPU
if gpu_devices:
    print("TensorFlow is using GPU.")
    # Specify which GPU to use (assuming you have at least one GPU)
    tf.config.experimental.set_visible_devices(gpu_devices[0], 'GPU')
else:
    print("TensorFlow is NOT using GPU.")

Available GPUs: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
TensorFlow is using GPU.


# Get Data

In [3]:
# use wget  to download data

import wget

In [4]:
training_url = "https://storage.googleapis.com/tensorflow-1-public/course2/week3/horse-or-human.zip"
validation_url = "https://storage.googleapis.com/tensorflow-1-public/course2/week3/validation-horse-or-human.zip"

# download
# wget.download(training_url, out="training.zip")
# wget.download(validation_url, out="validation.zip")

In [5]:
import zipfile

# unzip training
local_zip = "./training.zip"
zip_ref = zipfile.ZipFile(local_zip, "r")
zip_ref.extractall("./training")

# unzip training
local_zip = "./validation.zip"
zip_ref = zipfile.ZipFile(local_zip, "r")
zip_ref.extractall("./validation")

zip_ref.close()

In [6]:
import os

# train horse directory
train_horse_dir = os.path.join("./training/horses/")

# train human dir
train_human_dir = os.path.join("./training/humans/")

# val horse dir
val_horse_dir = os.path.join("./validation/horses/")

# val human dir
val_human_dir = os.path.join("./validation/humans/")

In [7]:
print(f"Number of train horses : {len(os.listdir(train_horse_dir))}")
print(f"Number of train humans : {len(os.listdir(train_human_dir))}")
print(f"Number of val horses : {len(os.listdir(val_horse_dir))}")
print(f"Number of val humans : {len(os.listdir(val_human_dir))}")

Number of train horses : 500
Number of train humans : 527
Number of val horses : 128
Number of val humans : 128


In [8]:
# plot image

pass

# Build Model

In [9]:
model = tf.keras.models.Sequential([
    # 1st convolutional layer
    tf.keras.layers.Conv2D(16, (3, 3), activation="relu", input_shape=(300, 300, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),

    # 2nd convolutional layer
    tf.keras.layers.Conv2D(32, (3, 3), activation="relu"),
    tf.keras.layers.MaxPooling2D(2, 2),

    # 3rd convolutional layer
    tf.keras.layers.Conv2D(64, (3, 3), activation="relu"),
    tf.keras.layers.MaxPooling2D(2, 2),

    # 4th convolutional layer
    tf.keras.layers.Conv2D(64, (3, 3), activation="relu"),
    tf.keras.layers.MaxPooling2D(2, 2),

    # 5th convolutional layer
    tf.keras.layers.Conv2D(64, (3, 3), activation="relu"),
    tf.keras.layers.MaxPooling2D(2, 2),

    # DNN
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512, activation="relu"),
    tf.keras.layers.Dense(1, activation="sigmoid")
])

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 298, 298, 16)      448       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 149, 149, 16)     0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 147, 147, 32)      4640      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 73, 73, 32)       0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 71, 71, 64)        18496     
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 35, 35, 64)       0

In [10]:
from tensorflow.keras.optimizers import RMSprop

model.compile(
    loss="binary_crossentropy",
    optimizer=RMSprop(learning_rate=0.001),
    metrics=["accuracy"]
)

# Data Preprocessing

In [11]:
# import image generator
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# scaling
train_datagen = ImageDataGenerator(rescale=1/255)
val_datagen = ImageDataGenerator(rescale=1/255)

# flow training images using train_datagen generator
train_generator = train_datagen.flow_from_directory(
    "./training/",
    target_size=(300, 300),
    batch_size=64,
    class_mode="binary"
)

# flow validation images
val_generator = val_datagen.flow_from_directory(
    "./validation/",
    target_size=(300, 300),
    batch_size=32,
    class_mode="binary"
)

Found 1027 images belonging to 2 classes.
Found 256 images belonging to 2 classes.


In [12]:
# training

history = model.fit(
    train_generator,
    steps_per_epoch=8,
    epochs=15,
    verbose=1,
    validation_data=val_generator,
    validation_steps=8
)

Epoch 1/15
8/8 [==============================] - 27s 2s/step - loss: 0.7617 - accuracy: 0.4844 - val_loss: 0.6421 - val_accuracy: 0.5469
Epoch 2/15
8/8 [==============================] - 7s 853ms/step - loss: 0.7965 - accuracy: 0.6680 - val_loss: 0.5539 - val_accuracy: 0.8789
Epoch 3/15
8/8 [==============================] - 6s 759ms/step - loss: 0.4683 - accuracy: 0.8184 - val_loss: 0.7294 - val_accuracy: 0.7930
Epoch 4/15
8/8 [==============================] - 5s 663ms/step - loss: 0.6489 - accuracy: 0.7949 - val_loss: 0.4041 - val_accuracy: 0.8555
Epoch 5/15
8/8 [==============================] - 6s 710ms/step - loss: 0.2853 - accuracy: 0.8965 - val_loss: 1.1051 - val_accuracy: 0.8125
Epoch 6/15
8/8 [==============================] - 6s 708ms/step - loss: 0.1899 - accuracy: 0.9297 - val_loss: 1.4165 - val_accuracy: 0.8086
Epoch 7/15
8/8 [==============================] - 5s 655ms/step - loss: 0.1208 - accuracy: 0.9512 - val_loss: 1.2035 - val_accuracy: 0.8516
Epoch 8/15
8/8 [======